In [1]:
import os
import glob
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from skimage.io import imread
import skimage
import ipywidgets as widgets
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

# %matplotlib notebook
%matplotlib ipympl

In [2]:
seg_types = ['LocalTopHat', 'Grid', 'Sabrina','GroundTruth']

dataset_root = "/globalscratch/users/n/s/nsayez/deepsun_bioblue/2013"
split = 'test'

image_folder = os.path.join(dataset_root, split, 'image')
gt_folders = {t : os.path.join(dataset_root, split, t) for t in seg_types}

# run_folder = "/home/ucl/elen/nsayez/bio-blueprints/outputs/2021-10-22/23-40-09_LTH_A100"
# run_folder = "/home/ucl/elen/nsayez/outputs/2021-10-31/02-00-49_Grid"
# run_folder = "/home/ucl/elen/nsayez/outputs/2021-10-31/02-01-00_LTH_Grid"
run_folders = { "LTH":"/home/ucl/elen/nsayez/bio-blueprints/outputs/2021-10-22/23-40-09_LTH_A100",
                   "Grid":"/home/ucl/elen/nsayez/outputs/2021-10-31/02-00-49_Grid",
                   "LTH_Grid":"/home/ucl/elen/nsayez/outputs/2021-10-31/02-01-00_LTH_Grid"}

# run_folder = "/home/ucl/elen/nsayez/outputs/2021-10-23/12-55-30_Grid"
# run_folder = "/home/ucl/elen/nsayez/outputs/2021-10-23/13-01-22_LTH_Grid"
# run_folder = "/home/ucl/elen/nsayez/outputs/2021-10-23/20-42-14_Sabrina"
# pred_folder = os.path.join(run_folder, 'predictions')
pred_folders = {t: os.path.join(run_folders[t] , 'predictions') for t in run_folders.keys()}
pred_lsts = {t : sorted(glob.glob(os.path.join(pred_folders[t], '*.png'))) for t in run_folders.keys()}

image_lst = sorted(glob.glob(os.path.join(image_folder, '*.FTS')))
gt_lst = sorted(glob.glob(os.path.join(gt_folders['GroundTruth'], '*.png')))
# gt_lst = sorted(glob.glob(os.path.join(gt_folders['LocalTopHat'], '*.png')))
# gt_lst = sorted(glob.glob(os.path.join(gt_folders['Grid'], '*.png')))
# gt_lst = sorted(glob.glob(os.path.join(gt_folders['Sabrina'], '*.png')))
# pred_lst = sorted(glob.glob(os.path.join(pred_folder, '*.png')))

print(len(image_lst))


# assert len(image_lst) == len(pred_lst)
# assert len(gt_lst) == len(pred_lst)



211


In [3]:
max_rows = 2
max_cols = 2

max_idx = len(gt_lst)

# print(train_img.shape,train_label.shape)


cmap_pred = cm.cool
cmap_pred = cmap_pred(range(255))
cmap_pred = ListedColormap([(0, 0, 0, 0), *cmap_pred])

cmap_gt = cm.autumn
cmap_gt = cmap_gt(range(255))
cmap_gt = ListedColormap([(0, 0, 0, 0), *cmap_gt])


def refresh(slider): 
    xlims0 = axes0[0].get_xlim()
    ylims0 = axes0[0].get_ylim()
    
    axes0[0].clear()
    
    test_img = imread(image_lst[idx_slider.value])
    test_label = imread(gt_lst[idx_slider.value])
    print(gt_lst[idx_slider.value])
    pred_label = imread(pred_lst[idx_slider.value])
    
    if img_cb.value:
        axes0[0].imshow(test_img, cmap="gray", interpolation="None")
    if gt_cb.value:
        axes0[0].imshow(test_label, cmap=cmap_gt, interpolation="None", alpha=.5)
        
#     if pred_cb.value:
#         axes0[0].imshow(pred_label, cmap=cmap_pred, interpolation="None", alpha=.5)
    
    for i, cb in enumerate(pred_cbs):
        if cb.value: 
            tmp = imread(pred_lsts[list(pred_folders.keys())[i]][idx_slider.value])
            axes0[0].imshow(tmp, cmap=cmap_pred, interpolation="None", alpha=.5)  
        
    if xlims0 != (0.0, 1.0):
        axes0[0].set_xlim(xlims0)
        axes0[0].set_ylim(ylims0)
    return

max_rows = 1
max_cols = 1

plt.ioff()
fig_widget0, axes0 = plt.subplots(nrows=max_rows, ncols=max_cols, figsize=(8,8))

try:
    len(axes0)
except TypeError:
    axes0 = [axes0]

plt.ion()
img_cb = widgets.Checkbox(value=True, description='Show img')
gt_cb = widgets.Checkbox(value=False, description='Show gt')
# pred_cb = widgets.Checkbox(value=False, description='Show Prediction')
pred_cbs = [widgets.Checkbox(value=False, description=f'Show {t}') for t in pred_folders.keys()]
idx_slider = widgets.IntSlider(value=0, min=0, max=max_idx-1, step=1, description="Image Index")


# Input image to predict
test_img = imread(image_lst[0])
#Target corresponding to input
test_label = imread(gt_lst[0])
#prediction
pred_label = imread(pred_lst[0])

axes0[0].imshow(test_img, cmap="gray", interpolation="None")

img_cb.observe(refresh, names='value')
gt_cb.observe(refresh, names='value')
for cb in pred_cbs:
    cb.observe(refresh, names='value')
# pred_cb.observe(refresh, names='value')
idx_slider.observe(refresh, names='value')

widgets.VBox([widgets.HBox([idx_slider, img_cb, gt_cb]+pred_cbs), fig_widget0.canvas])

TraitError: setting max < min

In [82]:
axes0[0].set_xlim([640,1035])
axes0[0].set_ylim([1135, 1340])
axes0[0].invert_yaxis()

In [12]:
# plt.style.available

# Compute IoU

In [4]:
from typing import Optional
import pytorch_lightning as pl
from torchmetrics import ConfusionMatrix
import torch
import tqdm


class IoU(ConfusionMatrix):
    def __init__(
        self,
        num_classes: int,
        threshold: float = 0.5,
        class_index: Optional[int] = None,
    ):
        super().__init__(
            num_classes,
            normalize=None,
            threshold=threshold,
            compute_on_step=True,
            dist_sync_on_step=False,
        )
        self.class_index = class_index

    def compute(self) -> torch.Tensor:
        cm = super().compute().to(torch.double)
        iou = cm.diag() / (cm.sum(dim=1) + cm.sum(dim=0) - cm.diag() + 1e-15)
        if self.class_index is None:
            return iou.mean()
        else:
            return iou[self.class_index]


In [26]:
models_folders = { "LTH":"/home/ucl/elen/nsayez/bio-blueprints/outputs/2021-10-22/23-40-09_LTH_A100",
                   "Grid":"/home/ucl/elen/nsayez/outputs/2021-10-31/02-00-49_Grid",
                   "LTH_Grid":"/home/ucl/elen/nsayez/outputs/2021-10-31/02-01-00_LTH_Grid"}


gt_folder = os.path.join(dataset_root, 'GroundTruth')

pred_folders = {t : models_folders[t] for t in models_folders.keys()}
pred_lsts = {t : sorted(glob.glob(os.path.join(pred_folders[t],'predictions' , '*.png'))) for t in models_folders.keys()}

# print(pred_lsts)

# build tensor for the whole reference set
gt_tensors = [] 
for i, path in enumerate(gt_lst[:]):
    np_array = np.array(imread(path))
    gt_tensors.append( torch.unsqueeze(torch.from_numpy(np_array), axis=0))
    
tot_gt = torch.cat(gt_tensors)
print(tot_gt.shape)
#########################################
LTH_tensors = [] 
for i, path in enumerate(pred_lsts['LTH'][:]):
    np_array = np.array(imread(path))
#     print(np_array.shape)
    LTH_tensors.append( torch.unsqueeze(torch.from_numpy(np_array.astype(np.uint8)), axis=0))

tot_LTH = torch.cat(LTH_tensors)
print(tot_LTH.shape)
#########################################
Grid_tensors = [] 
for i, path in enumerate(pred_lsts['Grid'][:]):
    np_array = np.array(imread(path))
#     print(np_array.shape)
    Grid_tensors.append( torch.unsqueeze(torch.from_numpy(np_array.astype(np.uint8)), axis=0))

tot_Grid = torch.cat(Grid_tensors)
print(tot_Grid.shape)
#########################################
LTHGrid_tensors = [] 
for i, path in enumerate(pred_lsts['LTH_Grid'][:]):
    np_array = np.array(imread(path))
#     print(np_array.shape)
    LTHGrid_tensors.append( torch.unsqueeze(torch.from_numpy(np_array.astype(np.uint8)), axis=0))

tot_LTHGrid = torch.cat(LTHGrid_tensors)
print(tot_LTHGrid.shape)

torch.Size([36, 2048, 2048])
torch.Size([36, 2048, 2048])
torch.Size([36, 2048, 2048])
torch.Size([36, 2048, 2048])


LTH

In [17]:
LTH_ious = {'0':[], '1': [], '2':[] }
for i in tqdm.tqdm(range(tot_gt.shape[0])):
    for c in range(3):
        iou = IoU(num_classes=3, class_index=c)
        res = iou(tot_LTH[i,:,:], tot_gt[i,:,:])
        LTH_ious[str(c)].append(res)
#         print(f'class {c} -> {res}')
# print(ious)

100%|████████████████████████████████████████████████████████████████████| 36/36 [02:55<00:00,  4.88s/it]


In [18]:
LTH_mean_ious = {t: np.mean(LTH_ious[t]) for t in LTH_ious.keys()}
print(LTH_mean_ious)

{'0': 0.9994255558845039, '1': 0.46166523830002326, '2': 0.41807050159951287}


In [42]:
LTH_cm = ConfusionMatrix(num_classes=3)
LTH_cm(tot_LTH[0,:,:], tot_gt[0,:,:])
cm = LTH_cm.compute()
(100*cm).mean(axis=1)

tensor([1.3946e+08, 2.9563e+05, 5.1900e+04])

In [19]:
# CMs are shaped [target_classes X pred_classes]
LTH_cms = []
# for i in tqdm.tqdm(range(2)):
for i in tqdm.tqdm(range(tot_gt.shape[0])):
    LTH_cm = ConfusionMatrix(num_classes=3, normalize='true')
    LTH_cm(tot_LTH[i,:,:], tot_gt[i,:,:]).to('cuda:0')
    cm = LTH_cm.compute()
#     print(cm)
#     print((cm.sum(dim=1) + cm.sum(dim=0) - cm.diag()).sum() )
    LTH_cms.append(cm)
    
LTH_cms = [torch.unsqueeze(item, 0) for item in LTH_cms.copy()]

 50%|██████████████████████████████████                                  | 18/36 [00:30<00:28,  1.60s/it]/home/ucl/elen/nsayez/miniconda3/envs/A100-bioblue/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: 3 nan values found in confusion matrix have been replaced with zeros.
  warnings.warn(*args, **kwargs)
100%|████████████████████████████████████████████████████████████████████| 36/36 [01:00<00:00,  1.67s/it]


In [43]:
print(LTH_cms[1].shape)
tot_LTH_CM = torch.cat(LTH_cms)
print(tot_LTH_CM.shape)
mean_LTH_CM = torch.mean(100*tot_LTH_CM,0)
std_LTH_CM = torch.std(100*tot_LTH_CM,0)
# print(mean_LTH_CM)
# print(std_LTH_CM)

LTH_CM_percents = [[0,0,0],[0,0,0],[0,0,0]]
for i in range(mean_LTH_CM.shape[0]):
#     print(i)
    for j in range(mean_LTH_CM.shape[1]):
#         print(j)
        LTH_CM_percents[i][j] = '{:.2f}'.format(mean_LTH_CM[i,j])
    
print(LTH_CM_percents)

torch.Size([1, 3, 3])
torch.Size([36, 3, 3])
[['99.95', '0.05', '0.00'], ['10.50', '82.37', '7.13'], ['8.75', '30.70', '57.77']]


GRID

In [21]:
Grid_ious = {'0':[], '1': [], '2':[] }
for i in tqdm.tqdm(range(tot_gt.shape[0])):
    for c in range(3):
        iou = IoU(num_classes=3, class_index=c)
        res = iou(tot_Grid[i,:,:], tot_gt[i,:,:])
        Grid_ious[str(c)].append(res)
#         print(f'class {c} -> {res}')
# print(ious)

100%|████████████████████████████████████████████████████████████████████| 36/36 [02:53<00:00,  4.81s/it]


In [22]:
Grid_mean_ious = {t: np.mean(Grid_ious[t]) for t in Grid_ious.keys()}
print(Grid_mean_ious)

{'0': 0.9971051985493143, '1': 0.16713670299456862, '2': 0.2961009397067897}


In [23]:
# CMs are shaped [target_classes X pred_classes]
Grid_cms = []
# for i in tqdm.tqdm(range(2)):
for i in tqdm.tqdm(range(tot_gt.shape[0])):
    Grid_cm = ConfusionMatrix(num_classes=3, normalize='true')
    Grid_cm(tot_Grid[i,:,:], tot_gt[i,:,:]).to('cuda:0')
    cm = Grid_cm.compute()
#     print(cm)
#     print((cm.sum(dim=1) + cm.sum(dim=0) - cm.diag()).sum() )
    Grid_cms.append(cm)
    
Grid_cms = [torch.unsqueeze(item, 0) for item in Grid_cms.copy()]

100%|████████████████████████████████████████████████████████████████████| 36/36 [00:57<00:00,  1.60s/it]


In [45]:
print(Grid_cms[1].shape)
tot_Grid_CM = torch.cat(Grid_cms)
print(tot_Grid_CM.shape)
mean_Grid_CM = torch.mean(100*tot_Grid_CM,0)
std_Grid_CM = torch.std(100*tot_Grid_CM,0)
# print(mean_Grid_CM)
# print(std_Grid_CM)

Grid_CM_percents = [[0,0,0],[0,0,0],[0,0,0]]
for i in range(mean_Grid_CM.shape[0]):
#     print(i)
    for j in range(mean_Grid_CM.shape[1]):
#         print(j)
        Grid_CM_percents[i][j] = '{:.2f}'.format(mean_Grid_CM[i,j])
    
print(Grid_CM_percents)

torch.Size([1, 3, 3])
torch.Size([36, 3, 3])
[['99.71', '0.28', '0.00'], ['1.83', '79.09', '19.08'], ['0.94', '34.69', '61.60']]


LTH + GRID

In [27]:
LTHGrid_ious = {'0':[], '1': [], '2':[] }
for i in tqdm.tqdm(range(tot_gt.shape[0])):
    for c in range(3):
        iou = IoU(num_classes=3, class_index=c)
        res = iou(tot_LTHGrid[i,:,:], tot_gt[i,:,:])
        LTHGrid_ious[str(c)].append(res)
#         print(f'class {c} -> {res}')
# print(ious)

100%|████████████████████████████████████████████████████████████████████| 36/36 [02:54<00:00,  4.84s/it]


In [28]:
LTHGrid_mean_ious = {t: np.mean(LTHGrid_ious[t]) for t in LTHGrid_ious.keys()}
print(LTHGrid_mean_ious)

{'0': 0.9992724310155298, '1': 0.42064785852554576, '2': 0.36716534020620395}


In [29]:
# CMs are shaped [target_classes X pred_classes]
LTHGrid_cms = []
# for i in tqdm.tqdm(range(2)):
for i in tqdm.tqdm(range(tot_gt.shape[0])):
    LTHGrid_cm = ConfusionMatrix(num_classes=3, normalize='true')
    LTHGrid_cm(tot_LTHGrid[i,:,:], tot_gt[i,:,:]).to('cuda:0')
    cm = LTHGrid_cm.compute()
#     print(cm)
#     print((cm.sum(dim=1) + cm.sum(dim=0) - cm.diag()).sum() )
    LTHGrid_cms.append(cm)
    
LTHGrid_cms = [torch.unsqueeze(item, 0) for item in LTHGrid_cms.copy()]

100%|████████████████████████████████████████████████████████████████████| 36/36 [00:57<00:00,  1.59s/it]


In [44]:
print(LTHGrid_cms[1].shape)
tot_LTHGrid_CM = torch.cat(LTHGrid_cms)
print(tot_Grid_CM.shape)
mean_LTHGrid_CM = torch.mean(100*tot_LTHGrid_CM,0)
std_LTHGrid_CM = torch.std(100*tot_LTHGrid_CM,0)
# print(mean_LTHGrid_CM)
# print(std_LTHGrid_CM)

LTHGrid_CM_percents = [[0,0,0],[0,0,0],[0,0,0]]
for i in range(mean_LTHGrid_CM.shape[0]):
#     print(i)
    for j in range(mean_LTHGrid_CM.shape[1]):
#         print(j)
        LTHGrid_CM_percents[i][j] = '{:.2f}'.format(mean_LTHGrid_CM[i,j])
    
print(LTHGrid_CM_percents)

torch.Size([1, 3, 3])
torch.Size([36, 3, 3])
[['99.93', '0.07', '0.00'], ['8.24', '82.03', '9.73'], ['3.92', '38.00', '55.30']]
